In [10]:
import datetime
import numpy as np
import os.path
import pandas as pd
import sched
import time
import yfinance as yf

tick_seconds = 20

def current_stock_df_name(stock_name):
    return "current_" + stock_name + ".csv"

def history_stock_df_name(timestamp, stock_name):
    return str(timestamp) + stock_name + ".csv"

requests_filename = "current_requests.csv"
stats_filename = "stats.csv"
def update_stocks_collect_stat(scheduler): 
    import_interval = "5m"
    period_of_import = "60m"
    cur_df = yf.download(tickers = "TSLA", period = period_of_import, interval = import_interval)
    iteration_timestamp = datetime.datetime.now()
    global tick_seconds, requests_filename, stats_filename
    
    requests = open(requests_filename, "r")
    stats = open(stats_filename, "a")
    for raw_row in requests:
        row = raw_row.split(",")
        stock_name = row[0]
        try:
            if (os.path.isfile(current_stock_df_name(stock_name))):
                cur_df = pd.read_csv(current_stock_df_name(stock_name))
                cur_df.to_csv(history_stock_df_name(iteration_timestamp, stock_name))
            cur_df = yf.download(tickers = stock_name, period = period_of_import, interval = import_interval)
            cur_df.to_csv(current_stock_df_name(stock_name))
        except:
            print(f"Error has occured")
        stats.write(str(iteration_timestamp) + ",")
        stats.write(raw_row)
    stats.write("\n")
    requests.close()
    stats.close()
    print("Iteration has finished")
    scheduler.enter(tick_seconds, 1, update_stocks_collect_stat, (scheduler,))

my_scheduler = sched.scheduler(time.time, time.sleep)
my_scheduler.enter(tick_seconds, 1, update_stocks_collect_stat, (my_scheduler,))
my_scheduler.run()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Iteration has finished
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Iteration has finished
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Iteration has finished


KeyboardInterrupt: 

In [8]:
cur_df = yf.download(tickers = "TSLA", period = "5d", interval = "60m")

[*********************100%%**********************]  1 of 1 completed
